# 3.3 线性回归的简单实现(Pytorch)

## 生成数据集

In [4]:
import torch
import numpy as np
num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2
features = torch.tensor(np.random.normal(0, 1, (num_examples, num_inputs)), dtype=torch.float)
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += torch.tensor(np.random.normal(0, 0.01, size=labels.size()), dtype=torch.float)

## 读取数据

In [6]:
import torch.utils.data as Data

batch_size = 10
# 将训练数据的特征和标签组合
dataset = Data.TensorDataset(features, labels)
data_iter = Data.DataLoader(dataset, batch_size, shuffle=True)

for x, y in data_iter:
    print(x, y)
    break

tensor([[-0.3136, -0.7965],
        [-0.3814,  0.5869],
        [-0.7988,  0.7971],
        [-0.5990, -1.0908],
        [ 0.3118, -1.2748],
        [-0.8141,  0.1080],
        [-1.7431, -1.0828],
        [-0.2758, -0.3928],
        [ 0.5262,  0.2893],
        [-0.3554,  1.1577]]) tensor([ 6.2763,  1.4363, -0.0964,  6.7016,  9.1546,  2.2155,  4.4002,  4.9621,
         4.2802, -0.4408])


## 定义模型
一个nn.Module实例应该包含一些层以及返回输出的前向传播（forward）方法

In [9]:
import torch.nn as nn
class LinearNet(nn.Module):
    def __init__(self, n_feature):
        super(LinearNet, self).__init__()
        self.linear = nn.Linear(n_feature, 1)
    def forward(self, x):
        y = self.linear(x)
        return y

net = LinearNet(num_inputs)
print(net)

LinearNet(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


还可以用nn.Sequential来更加方便地搭建网络，Sequential是一个有序的容器，网络层将按照在传入Sequential的顺序依次被添加到计算图中

In [10]:
# 写法一
net = nn.Sequential(
    nn.Linear(num_inputs, 1)
    # nn.conv2d .......
)

# 写法二
net = nn.Sequential()
net.add_module('linear', nn.Linear(num_inputs, 1))
# net.add_module

# 写法三
from collections import OrderedDict
net = nn.Sequential(OrderedDict([
    ('linear', nn.Linear(num_inputs, 1))
    # .....
]))

print(net)
print(net[0])

Sequential(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)
Linear(in_features=2, out_features=1, bias=True)


可以通过net.parameters()来查看模型所有的可学习参数

In [11]:
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[ 0.7048, -0.1957]], requires_grad=True)
Parameter containing:
tensor([-0.2624], requires_grad=True)


## 初始化模型参数

In [18]:
from torch.nn import init

# 通过init.normal_将权重参数每个元素初始化为随机采样于均值为0、标准差为0.01的正态分布
init.normal_(net.linear.weight, mean=0, std=0.01)
init.constant_(net.linear.bias, val=0)

Parameter containing:
tensor([0.], requires_grad=True)

## 定义损失函数

In [19]:
loss = nn.MSELoss()

## 定义优化算法

In [20]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.03)
print(optim)

<module 'torch.optim' from 'D:\\install\\conda\\lib\\site-packages\\torch\\optim\\__init__.py'>


In [21]:
# 调整学习率
for param_group in optimizer.param_groups:
    param_group['lr'] *= 0.1

## 训练模型

In [25]:
num_epochs = 10
for epoch in range(1, num_epochs + 1):
    for x, y in data_iter:
        output = net(x)
        l = loss(output, y.view(-1, 1))
        optimizer.zero_grad()
        l.backward()
        optimizer.step()
    print('epoch %d, loss: %f' % (epoch, l.item()))

epoch 1, loss: 0.340522
epoch 2, loss: 0.051101
epoch 3, loss: 0.013948
epoch 4, loss: 0.003558
epoch 5, loss: 0.001734
epoch 6, loss: 0.001063
epoch 7, loss: 0.000146
epoch 8, loss: 0.000070
epoch 9, loss: 0.000142
epoch 10, loss: 0.000052


In [26]:
dense = net[0]
print(true_w, dense.weight)
print(true_b, dense.bias)

[2, -3.4] Parameter containing:
tensor([[ 1.9996, -3.3996]], requires_grad=True)
4.2 Parameter containing:
tensor([4.1986], requires_grad=True)
